In [1]:
import boto3
import pandas as pd
import numpy as np
from datetime import datetime


In [3]:
## fetch gunshot credentials before running the notebook
dev = boto3.Session(profile_name='asu_cic_gunshot')
dynamodb = dev.resource(
    'dynamodb', 
)
triangulated_gunshots = dynamodb.Table('triangulated_gunshots')
uplink_data = dynamodb.Table('lora-sensor-uplink-data')

In [ ]:
bdone = False
start_key = None
tg_data = []
scan_kwargs = {}
while not done:
    if start_key:
        scan_kwargs['ExclusiveStartKey'] = start_key
    response = triangulated_gunshots.scan(**scan_kwargs)
    tg_data.extend(response.get('Items', []))
    start_key = response.get('LastEvaluatedKey',None)
    done = start_key is None
len(tg_data)

5701

In [ ]:
done = False
start_key = None
ul_data = []
scan_kwargs = {}
while not done:
    if start_key:
        scan_kwargs['ExclusiveStartKey'] = start_key
    response = uplink_data.scan(**scan_kwargs)
    ul_data.extend(response.get('Items', []))
    start_key = response.get('LastEvaluatedKey',None)
    done = start_key is None
len(ul_data)

6121

In [ ]:
tg_df = pd.DataFrame(tg_data)
tg_df.head()

,lat,long,timestamp
0,33.4325,-112.0955,1658727145550
1,33.4324,-112.0938,1657592664315
2,33.4316,-112.095,1658026485992
3,33.4325,-112.0955,1657579545903
4,33.4324,-112.0938,1657694067050


In [ ]:
ul_df = pd.DataFrame(ul_data)
ul_df = ul_df[['device_id', 'timestamp']]
ul_df.head()

,device_id,timestamp
0,100000001e6b841b,1656539975662
1,100000001e6b841b,1656540258591
2,100000001e6b841b,1656540267415
3,100000001e6b841b,1656540294628
4,100000001e6b841b,1656540306325


In [ ]:
# convert timestamps to datatime objects
tg_df['datetime'] = tg_df['timestamp'].apply(lambda x: datetime.fromtimestamp(int(x/1000)))
tg_df.drop('timestamp', axis=1, inplace=True)
tg_df.sort_values(by='datetime', inplace=True)
tg_df.head()

,lat,long,datetime
4934,33.4315,-112.0955,2022-06-29 12:07:58
2439,33.4323,-112.095,2022-06-29 12:30:15
2705,33.4323,-112.0945,2022-06-29 12:47:21
4998,33.4316,-112.095,2022-06-29 12:53:52
2468,33.433,-112.0956,2022-06-29 13:19:56


In [ ]:
# convert timestamps to datatime objects
ul_df['datetime'] = ul_df['timestamp'].apply(lambda x: datetime.fromtimestamp(int(x/1000)))
ul_df.drop('timestamp', axis=1, inplace=True)
ul_df.sort_values(by='datetime', inplace=True)
ul_df.head()

,device_id,datetime
71,10000000297f6420,2022-06-29 12:07:52
5386,10000000ad2b6f70,2022-06-29 12:30:09
5405,100000004990fab7,2022-06-29 12:47:15
798,10000000b6f84019,2022-06-29 12:53:46
2342,1000000037272de8,2022-06-29 13:19:50


In [ ]:
ul_df['interval'] = ul_df['datetime'] - ul_df['datetime'].shift()
ul_df['interval_seconds'] = ul_df['interval'].dt.total_seconds()
ul_df['different_devices'] = ul_df['device_id'] != ul_df['device_id'].shift()
ul_df.head()

,device_id,datetime,interval,interval_seconds,different_devices
71,10000000297f6420,2022-06-29 12:07:52,NaT,NaN,True
5386,10000000ad2b6f70,2022-06-29 12:30:09,0 days 00:22:17,1337.0,True
5405,100000004990fab7,2022-06-29 12:47:15,0 days 00:17:06,1026.0,True
798,10000000b6f84019,2022-06-29 12:53:46,0 days 00:06:31,391.0,True
2342,1000000037272de8,2022-06-29 13:19:50,0 days 00:26:04,1564.0,True


In [ ]:
# number of devices that reported within one second of each other
ul_df.query('interval_seconds <= 1 and different_devices == True')

,device_id,datetime,interval,interval_seconds,different_devices
5408,100000004990fab7,2022-06-29 14:48:13,0 days 00:00:00,0.0,True
2345,1000000037272de8,2022-06-29 15:04:19,0 days 00:00:01,1.0,True
801,10000000b6f84019,2022-06-29 15:04:54,0 days 00:00:00,0.0,True
802,10000000b6f84019,2022-06-29 15:05:06,0 days 00:00:00,0.0,True
807,10000000b6f84019,2022-06-29 15:06:33,0 days 00:00:01,1.0,True
...,...,...,...,...,...
697,10000000297f6420,2022-08-19 16:28:10,0 days 00:00:01,1.0,True
5314,10000000f4abc1e4,2022-08-23 17:47:39,0 days 00:00:00,0.0,True
5318,10000000f4abc1e4,2022-08-23 17:49:02,0 days 00:00:01,1.0,True
2201,10000000b6f84019,2022-08-27 14:17:24,0 days 00:00:00,0.0,True


In [ ]:
# number of devices that reported within one second of each other
ul_df.query('interval_seconds <= 2 and different_devices == True')

,device_id,datetime,interval,interval_seconds,different_devices
5408,100000004990fab7,2022-06-29 14:48:13,0 days 00:00:00,0.0,True
2345,1000000037272de8,2022-06-29 15:04:19,0 days 00:00:01,1.0,True
801,10000000b6f84019,2022-06-29 15:04:54,0 days 00:00:00,0.0,True
802,10000000b6f84019,2022-06-29 15:05:06,0 days 00:00:00,0.0,True
807,10000000b6f84019,2022-06-29 15:06:33,0 days 00:00:01,1.0,True
...,...,...,...,...,...
5318,10000000f4abc1e4,2022-08-23 17:49:02,0 days 00:00:01,1.0,True
5361,10000000f4abc1e4,2022-08-27 14:16:52,0 days 00:00:02,2.0,True
2201,10000000b6f84019,2022-08-27 14:17:24,0 days 00:00:00,0.0,True
759,10000000297f6420,2022-08-27 14:27:20,0 days 00:00:01,1.0,True


In [ ]:
pd.merge_asof(
        tg_df,
        ul_df,
        direction="nearest",
)

,lat,long,datetime,device_id
0,33.4315,-112.0955,2022-06-29 12:07:58,10000000297f6420
1,33.4323,-112.095,2022-06-29 12:30:15,10000000ad2b6f70
2,33.4323,-112.0945,2022-06-29 12:47:21,100000004990fab7
3,33.4316,-112.095,2022-06-29 12:53:52,10000000b6f84019
4,33.433,-112.0956,2022-06-29 13:19:56,10000000b6f84019
...,...,...,...,...
5696,33.4316,-112.095,2022-09-02 01:22:54,10000000b6f84019
5697,33.4315,-112.0955,2022-09-02 08:25:57,10000000297f6420
5698,33.4325,-112.0955,2022-09-02 12:01:14,10000000f4abc1e4
5699,33.4315,-112.0955,2022-09-02 12:05:41,10000000297f6420
